<a href="https://colab.research.google.com/github/ZhiyuanC04/Personal/blob/main/OpenDelta_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Efficient training with multitask-serving with OpenDelta

### PART 1
In this notebook, you will see three interesting task around a question provided by a user.
1. We load a Delta to correct the spelling in the question.

2. We load another Delta to recognize the question's topic.

3.  We load another Delta to answer the question.

All this functionality shares exactly the same backbone model: T5-large. You can learn how to use the OpenDelta for multitask-serving with 1/N GPU RAM (N is the number of tasks) compared to finetuning.

### PART2
Then you will learn how to use OpenDelta to train a t5-large using batchsize=32 with only 11G GPU RAM (Which is provided by google for free).
**（You need to connect to the free GPU on the upper-right of the page.)**

### PART3
You will see the flexibility that OpenDelta provides to customize a delta model.

In [ ]:
!nvidia-smi

Mon Jul  4 04:02:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# install necessary packages
!pip install transformers --quiet
!pip install datasets==2.0 --quiet
!pip install opendelta==0.2.2 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opendelta 0.2.2 requires datasets==1.17.0, but you have datasets 2.0.0 which is incompatible.


In [ ]:
from dataclasses import dataclass, field
from typing import Optional, List
from transformers import Seq2SeqTrainingArguments, TrainerCallback
from datasets import load_dataset, load_metric, concatenate_datasets
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    HfArgumentParser,
    MBartTokenizer,
    default_data_collator,
    set_seed,
)
from datasets import load_dataset
import torch
import numpy as np
import random

In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """
    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )

model_args = ModelArguments(model_name_or_path="t5-large", )

### Load the backbone model using the traditional ways.

In [ ]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
config.dropout_rate = 0.0
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Embedding(32100, 1024)

### PART0: Visualization
#### Using OpenDelta as a fantastic visualization tool.

In [ ]:
from opendelta import Visualization
Visualization(model).structure_graph();

root
├── shared(Embedding),lm_head(Linear) weight:[32100, 1024]
├── encoder (T5Stack)
│   ├── embed_tokens (Embedding) weight:[32100, 1024]
│   ├── block (ModuleList)
│   │   ├── 0 (T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   │   └── relative_attention_bias (Embedding) weight:[32, 16]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   └── 1-23(T5Block)
│   │       └── layer (ModuleList)
│   │           ├── 0 (T5LayerSelfAttention)
│   │           │   ├── SelfAttention (T5Attention)
│   │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
│   │           │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │           └── 1 (T5LayerFF)
│   │               ├── DenseReluDense (T5DenseActDense)
│   │               │   ├── wi (Linear) weight:[4096, 1024]
│   │               │   └── wo (Linear) weight:[1024, 4096]
│   │               └── layer_norm (T5LayerNorm) weight:[1024]
│   └── final_layer_norm (T5LayerNorm) weight:[1024]
└── decoder (T5Stack)
    ├── embed_tokens (Embedding) weight:[32100, 1024]
    ├── block (ModuleList)
    │   ├── 0 (T5Block)
    │   │   └── layer (ModuleList)
    │   │       ├── 0 (T5LayerSelfAttention)
    │   │       │   ├── SelfAttention (T5Attention)
    │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   │   └── relative_attention_bias (Embedding) weight:[32, 16]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       ├── 1 (T5LayerCrossAttention)
    │   │       │   ├── EncDecAttention (T5Attention)
    │   │       │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       └── 2 (T5LayerFF)
    │   │           ├── DenseReluDense (T5DenseActDense)
    │   │           │   ├── wi (Linear) weight:[4096, 1024]
    │   │           │   └── wo (Linear) weight:[1024, 4096]
    │   │           └── layer_norm (T5LayerNorm) weight:[1024]
    │   └── 1-23(T5Block)
    │       └── layer (ModuleList)
    │           ├── 0 (T5LayerSelfAttention)
    │           │   ├── SelfAttention (T5Attention)
    │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │           │   └── layer_norm (T5LayerNorm) weight:[1024]
    │           ├── 1 (T5LayerCrossAttention)
    │           │   ├── EncDecAttention (T5Attention)
    │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │           │   └── layer_norm (T5LayerNorm) weight:[1024]
    │           └── 2 (T5LayerFF)
    │               ├── DenseReluDense (T5DenseActDense)
    │               │   ├── wi (Linear) weight:[4096, 1024]
    │               │   └── wo (Linear) weight:[1024, 4096]
    │               └── layer_norm (T5LayerNorm) weight:[1024]
    └── final_layer_norm (T5LayerNorm) weight:[1024]

#### Compare to the original visualization in pytorch

In [ ]:
model

### PART1:  Multitask Serving with Finetuned Deltas.
Load the three special delta models with different functionalities.

In [ ]:
from opendelta import AutoDeltaConfig, AutoDeltaModel

delta_model_spelling = AutoDeltaModel.from_finetuned("thunlp/Spelling_Correction_T5_LRAdapter_demo", backbone_model=model)
delta_model_spelling.detach()

delta_model_topic = AutoDeltaModel.from_finetuned("thunlp/Question_Topic_T5-large_Compacter", backbone_model=model)
delta_model_topic.detach()

delta_model_fact = AutoDeltaModel.from_finetuned("thunlp/FactQA_T5-large_Adapter", backbone_model=model)
delta_model_fact.detach()


Reuse the cached checkpoint in /root/.cache/delta_center/Spelling_Correction_T5_LRAdapter_demo
[WARNING|(OpenDelta)delta_configs:195]2022-07-04 04:04:02,109 >> The following keys are not used by <class 'opendelta.delta_models.low_rank_adapter.LowRankAdapterConfig'>.__init__ function: ['delta_type', 'opendelta_version', 'transformers_version']
[INFO|(OpenDelta)delta_configs:209]2022-07-04 04:04:02,111 >> Model config
LowRankAdapterConfig {
  "backbone_checkpoint_name": "t5-large",
  "backbone_class": "T5ForConditionalGeneration",
  "backbone_hash": "baa014d5e81363ff48935ba78b5df374",
  "common_structure": null,
  "low_rank_rank": 1,
  "low_rank_w_init": "glorot-uniform",
  "modified_modules": [
    "SelfAttention",
    "DenseReluDense"
  ],
  "non_linearity": "gelu_new",
  "opendelta_version": "0.2.2",
  "reduction_factor": 32,
  "transformers_version": "4.20.1"
}

Reuse the cached checkpoint in /root/.cache/delta_center/Spelling_Correction_T5_LRAdapter_demo
[WARNING|(OpenDelta)saving_l

In [ ]:
def multitask_serving(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids#.cuda()
    delta_model_spelling.attach()
    answers_ids =model.generate(input_ids=input_ids, max_length=20, num_beams=4)
    input_text = tokenizer.decode(answers_ids[0], skip_special_tokens=True)
    print("Correct Spelling: {}".format(input_text))
    delta_model_spelling.detach()

    delta_model_topic.attach()
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids#.cuda()
    answers_ids =model.generate(input_ids=input_ids, max_length=20, num_beams=4)
    topic = tokenizer.decode(answers_ids[0], skip_special_tokens=True)
    delta_model_topic.detach()
    print("Question Topic: {}".format(topic))

    delta_model_fact.attach()
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids#.cuda()
    answers_ids =model.generate(input_ids=input_ids, max_length=20, num_beams=4)
    input_text = tokenizer.decode(answers_ids[0], skip_special_tokens=True)
    delta_model_fact.detach()
    print("Question Answer: {}".format(input_text))




In [ ]:
multitask_serving("When was Beiiing olymp#ic heldd ?")
multitask_serving("What the commmon career of Newton ad eintesin?")

Correct Spelling: When was Beijing Olympic held?
Question Topic: The question's topic is sports.
Question Answer: 2008
Correct Spelling: What was the common career of Newton and Einstein?
Question Topic: The question's topic is science.
Question Answer: Physicists


In [ ]:
delta_model_spelling.detach()
delta_model_topic.detach()
delta_model_fact.detach()

## PART2: Efficient Tuning a T5-large with Limited GPU RAM

###  spelling correction as an example

In [ ]:
@dataclass
class TrainingArguments(Seq2SeqTrainingArguments):
    print_num_parameters: Optional[bool] = field(default=False, metadata={"help": "If set, print the parameters of "
                                                                                 "the model."})
    do_test: Optional[bool] = field(default=False, metadata={"help": "If set, evaluates the test performance."})
    split_validation_test: Optional[bool] = field(default=False,
                                                  metadata={"help": "If set, for the datasets which do not"
                                                                    "have the test set, we use validation set as their"
                                                                    "test set and make a validation set from either"
                                                                    "splitting the validation set into half (for smaller"
                                                                    "than 10K samples datasets), or by using 1K examples"
                                                                    "from training set as validation set (for larger"
                                                                    " datasets)."})
    compute_time: Optional[bool] = field(default=False, metadata={"help": "If set measures the time."})
    compute_memory: Optional[bool] = field(default=False, metadata={"help": "if set, measures the memory"})


training_args = TrainingArguments(output_dir="./",
                                  do_train=True,
                                  do_eval=True,
                                  do_predict=False,
                                  evaluation_strategy="steps",
                                  eval_steps=200,
                                  save_strategy="steps",
                                  save_steps=200,
                                  greater_is_better=True,
                                  load_best_model_at_end=True,
                                  compute_memory=True,
                                  predict_with_generate=True,
                                  push_to_hub=False,
                                  learning_rate=1e-3,
                                  seed=42,
                                  per_device_eval_batch_size=32,
                                  per_device_train_batch_size=32,
                                  num_train_epochs=1,
                                  metric_for_best_model="em",
                                  warmup_steps=0,
                                  save_total_limit=1,
                                  gradient_accumulation_steps=1
                                  )

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = [tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=True) for i in preds]
    decoded_labels = [tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=True) for i in labels]
    result = {}
    result_list = [int(i==j) for i, j in zip(decoded_labels, decoded_preds)]
    result.update({"em":sum(result_list)/len(result_list)})
    return result

In [ ]:
mydataset = load_dataset("trivia_qa","unfiltered.nocontext")
mydataset['train'] = mydataset['train']
validation_index = np.arange(len(mydataset['validation']))
np.random.shuffle(validation_index)
mydataset['validation'] = mydataset['validation'].select(validation_index[:500])

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/633M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset trivia_qa downloaded and prepared to /root/.cache/huggingface/datasets/trivia_qa/unfiltered.nocontext/1.2.0/e73c5e47a8704744fa9ded33504b35a6c098661813d1c2a09892eb9b9e9d59ae. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

We randomly disturb the input question, for example change "Is it a good movie?" to "Ia it af movi good?" The target is the original sentences.

In [ ]:
def misspelling(x):
    length = len(x)
    replace_time = np.random.randint(3)
    count = 0

    while (count<replace_time):
        randfloat = np.random.rand()
        if randfloat < 0.15:
            x = x.split()
            switch_index = [np.random.randint(low=0, high=len(x)) for i in range(2)]
            tmp = x[switch_index[0]]
            x[switch_index[0]] = x[switch_index[1]]
            x[switch_index[1]] = tmp
            x = " ".join(x)
        elif randfloat < 0.3:
            x = x.split()
            drop_index = np.random.randint(low=0, high=len(x))
            x = x[:drop_index] + x[drop_index+1:]
            x = " ".join(x)
        elif randfloat < 0.8:
            replace_str = "".join([random.choice('abcdefghijklmnopqrstuvwxyz!@#$%^&*()') for i in range(np.random.randint(1,3))])
            rindx = np.random.randint(low=0, high=length)
            x = x[:rindx]+replace_str+x[rindx+1:]
        else:
            x=list(x)
            switch_index = [np.random.randint(low=0, high=len(x)) for i in range(2)]
            tmp = x[switch_index[0]]
            x[switch_index[0]] = x[switch_index[1]]
            x[switch_index[1]] = tmp
            x = "".join(x)
        count+=1
    return x

def tokenize_function(examples):
    input_sentences = [" ".join((i.strip("\n").strip().strip("?")+"?").split()[:20]) for i in examples["question"]]
    mis_spellings = [misspelling(x) for x in input_sentences]
    input_ids = [tokenizer.encode(i, padding="max_length", truncation=True, max_length=64) for i in mis_spellings]
    label = [tokenizer.encode(i, padding="max_length", truncation=True, max_length=64) for i in input_sentences]
    return {"input_ids": input_ids, "labels": label}

tokenized_datasets = mydataset.map(tokenize_function, remove_columns=['answer', 'question_source',"entity_pages",'search_results'], batched=True)

Parameter 'function'=<function tokenize_function at 0x7f0bb7ec3830> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

### Now let's apply a LowRankAdapterModel

In [ ]:
from opendelta import LowRankAdapterModel
delta_model1 = LowRankAdapterModel(backbone_model=model, modified_modules=['SelfAttention', "DenseReluDense"])
delta_model1.freeze_module(set_state_dict = True)
delta_model1.log(delta_ratio=True, trainable_ratio=True, visualization=True)


root
├── shared(Embedding),lm_head(Linear) weight:[32100, 1024]
├── encoder (T5Stack)
│   ├── embed_tokens (Embedding) weight:[32100, 1024]
│   ├── block (ModuleList)
│   │   ├── 0 (T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   │   ├── relative_attention_bias (Embedding) weight:[32, 16]
│   │   │       │   │   └── low_rank_adapter_1 (LowRankAdapter)
│   │   │       │   │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32]
│   │   │       │   │       │   b:[32]
│   │   │       │   │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] 
│   │   │       │   │           b:[1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   ├── wo (Linear) weight:[1024, 4096]
│   │   │           │   └── low_rank_adapter_1 (LowRankAdapter)
│   │   │           │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32]
│   │   │           │       │   b:[32]
│   │   │           │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] 
│   │   │           │           b:[1024]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   └── 1-23(T5Block)
│   │       └── layer (ModuleList)
│   │           ├── 0 (T5LayerSelfAttention)
│   │           │   ├── SelfAttention (T5Attention)
│   │           │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
│   │           │   │   └── low_rank_adapter_1 (LowRankAdapter)
│   │           │   │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32]
│   │           │   │       │   b:[32]
│   │           │   │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] 
│   │           │   │           b:[1024]
│   │           │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │           └── 1 (T5LayerFF)
│   │               ├── DenseReluDense (T5DenseActDense)
│   │               │   ├── wi (Linear) weight:[4096, 1024]
│   │               │   ├── wo (Linear) weight:[1024, 4096]
│   │               │   └── low_rank_adapter_1 (LowRankAdapter)
│   │               │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32]
│   │               │       │   b:[32]
│   │               │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] 
│   │               │           b:[1024]
│   │               └── layer_norm (T5LayerNorm) weight:[1024]
│   └── final_layer_norm (T5LayerNorm) weight:[1024]
└── decoder (T5Stack)
    ├── embed_tokens (Embedding) weight:[32100, 1024]
    ├── block (ModuleList)
    │   ├── 0 (T5Block)
    │   │   └── layer (ModuleList)
    │   │       ├── 0 (T5LayerSelfAttention)
    │   │       │   ├── SelfAttention (T5Attention)
    │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   │   ├── relative_attention_bias (Embedding) weight:[32, 16]
    │   │       │   │   └── low_rank_adapter_1 (LowRankAdapter)
    │   │       │   │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32]
    │   │       │   │       │   b:[32]
    │   │       │   │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] 
    │   │       │   │           b:[1024]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       ├── 1 (T5LayerCrossAttention)
    │   │       │   ├── EncDecAttention (T5Attention)
    │   │       │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       └── 2 (T5LayerFF)
    │   │           ├── DenseReluDense (T5DenseActDense)
    │   │           │   ├── wi (Linear) weight:[4096, 1024]
    │   │           │   ├── wo (Linear) weight:[1024, 4096]
    │   │           │   └

[INFO|(OpenDelta)basemodel:675]2022-07-03 16:09:26,458 >> Trainable Ratio: 0.041213%
[INFO|(OpenDelta)basemodel:677]2022-07-03 16:09:26,461 >> Delta Parameter Ratio: 0.041213%
[INFO|(OpenDelta)basemodel:679]2022-07-03 16:09:26,462 >> Static Memory 0.00 GB, Max Memory 0.00 GB


In [ ]:
class MyCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        """
        Event called after an evaluation phase.
        """
        sents = ["was Wher Newton bon?",
                 "In year which Beiiing Olmpic ld?"
                ]
        for sent in sents:
            input_ids = tokenizer(sent, return_tensors="pt").input_ids.cuda()
            answers_ids =model.generate(input_ids=input_ids,
                                    max_length=20,
                                    num_beams=4,
                                    )
            print("{} {}".format(sent, tokenizer.decode(answers_ids[0], skip_special_tokens=True)))
        print("max allocated memory {} GB".format(torch.cuda.max_memory_allocated(f"cuda:0")/1024**3))

from transformers import Seq2SeqTrainer
training_args.output_dir = "./SpellingCorrection" # to avoid conflict
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    callbacks=[MyCallback],
    compute_metrics=compute_metrics,
)

trainer.train()

# delta_model.save_finetuned("Spelling_T5-lowrankadapter", push_to_hub=True)
# Since we are in a notebook without login in huggingface model hub, we omit this step.

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: question, question_id. If question, question_id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 87622
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2739


Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

## PART3: Examples of Customized Deltas

Here we add Adapters to 0\~5 layers' attention module, and 10\~12 layers' feed-forward module (after layer_norm).

We then add Lora to 0~9 layers' attention's Q matrix.

In [ ]:

from opendelta import AdapterModel
from opendelta import LoraModel
delta_model1.detach()
model.to("cpu")
delta_model_custom1 = AdapterModel(backbone_model=model,
                                   modified_modules=['[r][0-5]\.layer\.0\.SelfAttention', '[r](10|11|12)\.layer\.1\.layer_norm'])
delta_model_custom2 = LoraModel(backbone_model=model, modified_modules=['[r][0-9]\.layer\.0\.SelfAttention\.q'])
delta_model_custom2.freeze_module(set_state_dict = True)
delta_model_custom2.log(delta_ratio=True, trainable_ratio=True, visualization=True)

# delta_model_custom2.detach()

root
├── shared(Embedding),lm_head(Linear) weight:[32100, 1024]
├── encoder (T5Stack)
│   ├── embed_tokens (Embedding) weight:[32100, 1024]
│   ├── block (ModuleList)
│   │   ├── 0 (T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q (Linear) weight:[1024, 1024]
│   │   │       │   │   │   └── lora,lora_1(LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
│   │   │       │   │   ├── k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   │   ├── relative_attention_bias (Embedding) weight:[32, 16]
│   │   │       │   │   └── adapter_1,adapter_2(AdapterLayer)
│   │   │       │   │       └── modulelist (Sequential)
│   │   │       │   │           ├── down_proj (Linear) weight:[24, 1024] bias:[24]
│   │   │       │   │           └── up_proj (Linear) weight:[1024, 24] bias:[1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   ├── 1-5(T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q (Linear) weight:[1024, 1024]
│   │   │       │   │   │   └── lora,lora_1(LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
│   │   │       │   │   ├── k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   │   └── adapter_1,adapter_2(AdapterLayer)
│   │   │       │   │       └── modulelist (Sequential)
│   │   │       │   │           ├── down_proj (Linear) weight:[24, 1024] bias:[24]
│   │   │       │   │           └── up_proj (Linear) weight:[1024, 24] bias:[1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   ├── 6-9(T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q (Linear) weight:[1024, 1024]
│   │   │       │   │   │   └── lora,lora_1(LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
│   │   │       │   │   └── k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   ├── 10-12(T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │               └── adapter,adapter_1(AdapterLayer)
│   │   │                   └── modulelist (Sequential)
│   │   │                       ├── down_proj (Linear) weight:[24, 1024] bias:[24]
│   │   │                       └── up_proj (Linear) weight:[1024, 24] bias:[1024]
│   │   └── 13-23(T5Block)
│   │       └── layer (ModuleList)
│   │           ├── 0 (T5LayerSelfAttention)
│   │           │   ├── SelfAttention (T5Attention)
│   │           │   │   └── q,k,v,o(Linear) weight:[1024, 102

[INFO|(OpenDelta)basemodel:675]2022-07-03 16:11:11,718 >> Trainable Ratio: 0.332733%
[INFO|(OpenDelta)basemodel:677]2022-07-03 16:11:11,719 >> Delta Parameter Ratio: 0.332733%
[INFO|(OpenDelta)basemodel:679]2022-07-03 16:11:11,720 >> Static Memory 0.00 GB, Max Memory 0.00 GB
